In [ ]:
import sys
!{sys.executable} -m pip install awswrangler 

# S3 Select

Download some sample data sets and upload them to zgw-dbstore

In [ ]:
import awswrangler as swr
import pandas

example_csv = "http://bd-dist.s3.amazonaws.com/fhv_tripdata_2019-09.csv"

# Need to fetch and upload the parquet file, pyarrow does not support url sources
# curl -LO https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2022-01.parquet

target = "s3://mybucket/fhv_tripdata_2019-09.csv"

df = pandas.read_csv(example_csv)

wr.config.s3_endpoint_url = 'http://s3.default.svc.cluster.local'
wr.config.addressing_style = 'path'

wr.s3.to_csv(df, target, index=False)


## Read full CSV file

The works with Ceph

In [ ]:
df = wr.s3.select_query(
    sql="SELECT * FROM s3object",
    path="s3://mybucket/fhv_tripdata_2019-09.csv",
    input_serialization="CSV",
    input_serialization_params={
        "FileHeaderInfo": "IGNORE",
        "RecordDelimiter": "\r\n"
    },
    use_threads=True
)

This does not appear to work with Ceph, but works with Amazon S3

In [ ]:
df = wr.s3.select_query(
    sql="SELECT * FROM s3object",
    path="s3://mybucket/fhv_tripdata_2019-09.csv",
    input_serialization="CSV",
    input_serialization_params={
        "FileHeaderInfo": "Use",
        "RecordDelimiter": "\r\n"
    },
    use_threads=True,
)
df.head()

# Read Parquet

This does not appear to work with Ceph

In [ ]:
df = wr.s3.select_query(
        sql="SELECT * FROM s3object",
        path="s3://mybucket/fhv_tripdata_2022-01.parquet",
        input_serialization="Parquet",
        input_serialization_params={},
        use_threads=True,
)